In [ ]:
# Store Support Vector Machine data and Scaler to pickle file
print('Saving SVM and Scaler function to pickle file... ', end='')
datadump = {
    'svc': svc,
    'X_scaler': X_scaler,
    'colorspace': HP.colorspace,
    'orient': HP.HOG_orient,
    'pix_per_cell': HP.pix_per_cell,
    'cell_per_block': HP.cell_per_block,
    'hist_bins': HP.hist_bins,
    'spatial_size': HP.spatial_size,
    'block_norm': HP.block_norm,
    'transform_sqrt': HP.transform_sqrt    
    }
with open('./svm_scaler_result.pkl','wb') as f:
    pickle.dump(datadump,f)
print('DONE!')

In [ ]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler,
                    color_space='RGB', 
                    spatial_size=(32, 32),
                    hist_bins=32, 
                    hist_range=(0, 256),
                    orient=9, 
                    pix_per_cell=8,
                    cell_per_block=2, 
                    hog_channel=0,
                    spatial_feat=True, 
                    hist_feat=True,
                    hog_feat=True,
                    transform_sqrt=True,
                    block_norm='L2-Hys'):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img,
                                       cspace=color_space,
                                       orient=orient, vis=False,
                                       pix_per_cell=pix_per_cell,
                                       cell_per_block=cell_per_block,
                                       hog_channel=hog_channel, 
                                       spatial_feat=spatial_feat,
                                       hist_bins = hist_bins,
                                       spatial_size = spatial_size,
                                       hist_feat=hist_feat,
                                       hog_feat=hog_feat,
                                       transform_sqrt = transform_sqrt,
                                       block_norm = block_norm)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows


# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy    

    
# Heat Map helper function
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
    
# Heat map threshold function
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


# Draw labeled boxes
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img


def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [ ]:
def detect_cars(img,HP,svc,X_scaler,scale):
    # Initialize variables
    count = 0
    img_boxes = []
    window = 64 # Original window size
    # Prepare output image and heatmap variables
    
    draw_img = np.copy(img)
    draw_img2 = np.copy(img)
    heatmap = np.zeros_like(img[:,:,0])
    # Have it scaled appropriately to have colorspace match the png training data
    img = img.astype(np.float32)/255
    
    # Reduce search area
    img_sub = img[HP.y_lim[0]:HP.y_lim[1],:,:]
    
    # apply color conversion if other than 'RGB' is desired
    img_sub = color_conversion(img_sub, HP.colorspace)
    
    # Scale image
    if scale != 1:
        imshape = img_sub.shape
        img_s_scld = cv2.resize(img_sub, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        #print("Scaled sub image from shape",imshape,"to",img_s_scld.shape)
    else:
        img_s_scld = np.copy(img_sub)
    imshape = img_s_scld.shape

    # Define blocks and steps
    nxblocks = (imshape[1] // HP.pix_per_cell) - 1
    nyblocks = (imshape[0] // HP.pix_per_cell) - 1
    nfeat_per_block = HP.HOG_orient*HP.cell_per_block**2
    
    nblocks_per_window = (window // HP.pix_per_cell) - 1
    nxsteps = (nxblocks - nblocks_per_window) // HP.cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // HP.cells_per_step
    
    # Calculate hog channels for the entire scaled sub image
    hog1 = get_hog_features(img_s_scld[:,:,0], HP.HOG_orient, HP.pix_per_cell, HP.cell_per_block, feature_vec=False)
    hog2 = get_hog_features(img_s_scld[:,:,1], HP.HOG_orient, HP.pix_per_cell, HP.cell_per_block, feature_vec=False)
    hog3 = get_hog_features(img_s_scld[:,:,2], HP.HOG_orient, HP.pix_per_cell, HP.cell_per_block, feature_vec=False)

    for xb in range(nxsteps):
        for yb in range(nysteps):
            count += 1
            ypos = yb*HP.cells_per_step
            xpos = xb*HP.cells_per_step
            hogfeat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hogfeat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hogfeat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_features = np.hstack((hogfeat1,hogfeat2,hogfeat3))
    
            xleft = xpos*HP.pix_per_cell
            ytop = ypos*HP.pix_per_cell
            
            # Extract the image patch
            subimg = cv2.resize(img_s_scld[ytop:ytop+window, xleft:xleft+window],(64,64))
            
            # Get color features
            spatial_features = get_spatial_features(subimg, size=HP.spatial_size)
            hist_features = get_histogram_features(subimg, nbins=HP.hist_bins)
            
            # Scale features and make a prediction
            all_features = np.hstack((hog_features,spatial_features, hist_features))
            
            test_features = X_scaler.transform(all_features.reshape(1,-1))
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,
                              (xbox_left,ytop_draw+HP.y_lim[0]),
                              (xbox_left+win_draw,ytop_draw+win_draw+HP.y_lim[0]),
                              (0,0,255),
                              6)
                img_boxes.append(((xbox_left,ytop_draw+HP.y_lim[0]),(xbox_left+win_draw,ytop_draw+win_draw+HP.y_lim[0])))
                heatmap[ytop_draw+HP.y_lim[0]:ytop_draw+win_draw+HP.y_lim[0], xbox_left:xbox_left+win_draw] += 1

            xbox_left = np.int(xleft*scale)
            ytop_draw = np.int(ytop*scale)
            win_draw = np.int(window*scale)
            cv2.rectangle(draw_img2,
                          (xbox_left,ytop_draw+HP.y_lim[0]),
                          (xbox_left+win_draw,ytop_draw+win_draw+HP.y_lim[0]),
                          (np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255)),
                          6)

    return (draw_img,heatmap,img_boxes,count,draw_img2)

In [ ]:
def process_image(image,):
    global buffer
#     draw_img1, heatmap1, img_boxes1, count1, draw_img1_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[0])
#     draw_img2, heatmap2, img_boxes2, count2, draw_img2_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[1])
#     draw_img3, heatmap3, img_boxes3, count3, draw_img3_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[2])
#     heatmap = heatmap1+heatmap2+heatmap3
    draw_img1, heatmap1, img_boxes1, count1, draw_img1_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[0])
    draw_img2, heatmap2, img_boxes2, count2, draw_img2_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[1])
    draw_img3, heatmap3, img_boxes3, count3, draw_img3_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[2])
    draw_img4, heatmap4, img_boxes4, count4, draw_img4_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[2])
    draw_img5, heatmap5, img_boxes5, count5, draw_img5_2 = detect_cars(image.astype(np.float32)/255,HP,svc,X_scaler,scale[2])
    heatmap = heatmap1+heatmap2+heatmap3+heatmap4+heatmap5

    # Store heatmap in double ended queue buffer, calculate averaged heatmap over 10 frames (i.e. buffer size)
    buffer.append(heatmap)
    heatmap_avg = np.average(buffer,0)
    heatmap_avg = apply_threshold(heatmap_avg, np.max(heatmap_avg)*0.2)
    heatmap_avg = apply_threshold(heatmap_avg, 3)

    # Detect labels for individual vehicles
    labels = label(heatmap_avg)
    
    return draw_labeled_bboxes(np.copy(image),labels)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import deque

# Initialize buffer for heatmap, define it as global variable
buffer = deque(maxlen=10)
global buffer

# Process video file
test_output = 'test.mp4'
# clip = VideoFileClip('test_video.mp4')
clip = VideoFileClip('project_video.mp4')
test_clip = clip.fl_image(process_image)
test_clip.write_videofile(test_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_output))